# Data Processing

## Remove Rows and Sample Randomly

We will now drop any rows that are irrelevant to us and save it to a CSV file.

In [2]:
import pandas as pd

# load data
df = pd.read_csv('data/Books_rating.csv')

# only preserve 'review/summary', 'review/text', and 'review/score' columns
df = df[['review/summary', 'review/text', 'review/score']]

# rename columns
df.columns = ['summary', 'text', 'score']

# choose 1 million random rows
df = df.sample(n=1000000, random_state=1)

# save to new csv file
df.to_csv('data/Books_rating_relevant_columns.csv', index=False)

# print first 5 rows
df.head()

,summary,text,score
2896109,Best edition of this classic.,I've always recommended this Yale edition of F...,5.0
2381153,Great Book!!,This is required reading for my 16 yr old son....,5.0
1028690,Not just a book for consultant,"Plain-spoken, finished the book only has taken...",4.0
1945977,Outrageously Bad,Wow... this is one of the most ridiculous stor...,1.0
2812693,Cunning and determination,A crew has mutinied and threatens to hang thei...,4.0


## Stopwords, Lemmatization, and Vectorization

Process the text to remove stopwords, lemmatize, strip unneeded characters, then vectorize.

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm

tqdm.pandas()

lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

def stemmer(text):
  if text != text:
    return ''
  
  # replace non-alphanumeric characters with space
  text = re.sub(r'[^a-zA-Z0-9]', ' ', text)

  # remove multiple spaces
  text = re.sub(' +', ' ', text)

  # # lowercase
  text = text.lower()

  # tokenize
  tokens = word_tokenize(text)

  # remove stopwords
  tokens = [w for w in tokens if not w in stopwords]

  # lemmatize
  tokens = [lemmatizer.lemmatize(w) for w in tokens]

  # join tokens
  text = ' '.join(tokens)

  return text

# load df
df = pd.read_csv('data/Books_rating_relevant_columns.csv')

# apply stemmer to text
df['stemmed_text'] = df['text'].progress_apply(stemmer)
df['stemmed_summary'] = df['summary'].progress_apply(stemmer)
df['stemmed_summary_text'] = df['stemmed_summary'] + ' ' + df['stemmed_text']

# remove trailing spaces
df['stemmed_summary_text'] = df['stemmed_summary_text'].progress_apply(lambda x: x.strip())

# replace NaN with empty string
df.fillna('', inplace=True)

# remove rows with empty stemmed_summary_text
df = df[df['stemmed_summary_text'] != '']

# drop unused columns
df.drop(columns=['text', 'summary', 'stemmed_summary', 'stemmed_text'], inplace=True)

# save to new csv file
df.to_csv('data/Books_rating_stemmed.csv', index=False)

# print first 5 rows
df.head()

100%|██████████| 1000000/1000000 [00:00<00:00, 2907145.14it/s]


,score,stemmed_summary_text
0,5.0,best edition classic always recommended yale e...
1,5.0,great book required reading 16 yr old son book...
2,4.0,book consultant plain spoken finished book tak...
3,1.0,outrageously bad wow one ridiculous story ever...
4,4.0,cunning determination crew mutinied threatens ...


## Vectorizing

Vectorize the text using TF-IDF.

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle

tqdm.pandas()

df = pd.read_csv('data/Books_rating_stemmed.csv')
print(df.shape)

vectorizer = TfidfVectorizer(max_features=30_000, sublinear_tf=True)

# create train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['stemmed_summary_text'], df['score'], test_size=0.2, random_state=1)

# fit vectorizer on training set
vectorizer.fit(X_train)

# transform training and test sets
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

# save vectorizer to disk
pickle.dump(vectorizer, open('models/vectorizer.pkl', 'wb'))
print('Vectorizer saved')

# get shape of training and test sets
print(X_train.shape)
print(X_test.shape)

(999999, 2)
Vectorizer saved
(799999, 30000)
(200000, 30000)


## Model Testing

Test the model using a variety of classifiers. For now, just Logistic Regression.

In [11]:
# train using logistic regression
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression(max_iter=1000, n_jobs=-1, solver='saga')
logistic_regression.fit(X_train, y_train)
print('Logistic regression trained')

# get accuracy, mse, and rmse
from sklearn.metrics import accuracy_score, mean_squared_error

y_pred = logistic_regression.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print('Accuracy:', accuracy)
print('MSE:', mse)
print('RMSE:', rmse)

Logistic regression trained
Accuracy: 0.694525
MSE: 0.837745
RMSE: 0.9152841088973412
